In [99]:
import os
from os.path import isfile
import nibabel as nb
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from scipy.stats import norm
import pickle
from nilearn import plotting
from scipy import ndimage
from kernel import kernel_conv
from compile_studies import compile_studies
from template import prior, affine, shape, pad_shape
from main_effect import compute_ale, compute_cnull, compute_z, compute_cluster, compute_tfce, compute_null

THRESH = 0.001

cwd = os.getcwd()
raw_folder = f'{cwd}/DataRaw/'
pickle_folder = f'{cwd}/DataPickle/'

'''filename = askopenfilename()
df = pd.read_excel(filename, engine='openpyxl', header=None)'''

meta_df = pd.read_excel(raw_folder + 'Metastocalculate.xlsx', engine='openpyxl', header=None)
with open(pickle_folder + 'experiments.pickle', 'rb') as f:
    exp_all, tasks = pickle.load(f)

row_idx = 3

exp_name1 = meta_df.iloc[row_idx, 1]
exp_idxs1 = compile_studies(meta_df, row_idx, tasks)
exp_df1 = exp_all.loc[exp_idxs1].reset_index(drop=True)
s1 = list(range(exp_df1.shape[0]))

exp_name2 = meta_df.iloc[row_idx + 1, 1]
exp_idxs2 = compile_studies(meta_df, row_idx+1, tasks)
exp_df2 = exp_all.loc[exp_idxs2].reset_index(drop=True)
s2 = list(range(exp_df2.shape[0]))

In [100]:
cwd = os.getcwd()
mask_folder = f"{cwd}/MaskenEtc/"
try:
    os.makedirs(f"{cwd}/ALE/rDiffs/Images")
    os.makedirs(f"{cwd}/ALE/rDiffs/Null")
except:
    pass

sample_n = 2500
target_n = 17

sample_space = np.array(np.where(prior == True))
peaks = np.array([exp_df1.XYZ[i].T[:,:3] for i in s0], dtype=object)
num_peaks = exp_df1.Peaks
kernels = exp_df1.Kernel

In [129]:
def create_samples(s0, sample_space, sample_n, target_n):
    samples = np.zeros((sample_n, target_n))
    s_perm = s0.copy()
    for i in range(sample_n):
        np.random.shuffle(s_perm)
        samples[i,:] = s_perm[:target_n]
    samples = np.unique(samples, axis=0)
    unique_samples = samples.shape[0]
    while unique_samples < sample_n:
        np.random.shuffle(s_perm)
        samples = np.vstack((samples, s_perm[:target_n]))
        samples = np.unique(samples, axis=0)
        unique_samples = samples.shape[0]
    return samples

In [3]:
# getClusterNull

uc = 0.001
s0 = exp_df1.shape[0]

s0 = list(range(exp_df1.shape[0]))
# highest possible ale value if every study had a peak at the same location.
mb = 1
for i in s0:
    mb = mb*(1-np.max(exp_df1.at[i, 'Kernel']))

# define bins for histogram
bin_edges = np.arange(0.00005,1-mb+0.001,0.0001)
bin_centers = np.arange(0,1-mb+0.001,0.0001)

In [9]:
max_ale, max_cluster = zip(*Parallel(n_jobs=3, verbose=2)(delayed(compute_null)(s0 = s1,
                                                                                 sample_space = sample_space,
                                                                                 num_peaks = num_peaks,
                                                                                 kernels = kernels,
                                                                                 bin_centers = bin_centers,
                                                                                 bin_edges = bin_edges,
                                                                                 c_null ="simulate") for i in range(1000)))

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    3.4s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:    5.0s
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    5.8s
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:    7.9s
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:    9.3s
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:   11.6s
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   14.0s
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:   17.1s
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:   19.5s
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:   23.1s
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:   26.0s
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed:   30.0s
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed:   33.6s
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed:   38.3s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:   41.8s
[Parallel(